In [ ]:
import pandas as pd
import numpy as np

import os
import io
import base64
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from fastai.vision.all import *
import torch.nn.functional as nnf

In [ ]:
cuda0 = torch.device('cuda:0')

In [ ]:
data_path = Path('Z:\cloud\data\hpa-single-cell-image-classification')
path = Path('../data')

In [ ]:
df = pd.read_csv(data_path/'train.csv')

In [ ]:
df.head()

In [ ]:
labels = [str(i) for i in range(19)]
for x in labels:
    df[x] = df['Label'].apply(lambda r: int(x in r.split('|')))

In [ ]:
dfs_0 = df[df['Label'] == '0'].sample(n=300, random_state=42).reset_index(drop=True)
dfs_1 = df[df['1'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_1u = df[df['Label'] == '1'].sample(n=221, random_state=42).reset_index(drop=True)
dfs_2 = df[df['Label'] == '2'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_3 = df[df['Label'] == '3'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_4 = df[df['Label'] == '4'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_5 = df[df['Label'] == '5'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_6 = df[df['6'] == 1].sample(n=500, random_state=42).reset_index(drop=True)
dfs_7 = df[df['Label'] == '7'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_8 = df[df['Label'] == '8'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_9 = df[df['9'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_9u = df[df['Label'] == '9'].sample(n=200, random_state=42).reset_index(drop=True)
dfs_10 = df[df['10'] == 1].sample(n=400, random_state=42).reset_index(drop=True)
dfs_10u = df[df['Label'] == '10'].sample(n=200, random_state=42).reset_index(drop=True)
dfs_11 = df[df['11'] == 1].reset_index(drop=True)
dfs_12 = df[df['Label'] == '12'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_13 = df[df['Label'] == '13'].sample(n=400, random_state=42).reset_index(drop=True)
dfs_14 = df[df['Label'] == '14'].sample(n=500, random_state=42).reset_index(drop=True)
dfs_15 = df[df['15'] == 1].reset_index(drop=True)
dfs_16 = df[df['Label'] == '16'].sample(n=350, random_state=42).reset_index(drop=True)
dfs_17 = df[df['17'] == 1].sample(n=500, random_state=42).reset_index(drop=True)
dfs_18 = df[df['18'] == 1].reset_index(drop=True)
dfs_ = [dfs_0, dfs_1, dfs_1u, dfs_2, dfs_3, dfs_4, dfs_5, dfs_6, dfs_7, dfs_8, dfs_9, dfs_9u, dfs_10, dfs_10u,
        dfs_11, dfs_12, dfs_13, dfs_14, dfs_15, dfs_16, dfs_17, dfs_18]

In [ ]:
dfs = pd.concat(dfs_, ignore_index=True)
dfs.drop_duplicates(inplace=True, ignore_index=True)

In [ ]:
def read_img(image_id: str, color: str, image_size: int):
    filename=f'{data_path}/train/{image_id}_{color}.png'
    img = Image.open(filename)

    size = (image_size, image_size)

    img = allPil2tensor2(img).to(cuda0)

    img = img.unsqueeze(0)
    img = img.float()
    img = nnf.interpolate(img, size=size, mode='bilinear', align_corners=False)
    img = img.type(torch.uint8)
    img = img[0,:,:,:]

    return img

def allPil2tensor2(image: Image.Image)->TensorImage:
    arr = np.asarray(image)

    if arr.ndim==2 : arr = np.expand_dims(arr,2)

    # Transpose width, height to height,width
    arr = np.transpose(arr, (1,0,2))

    # Move channels to the first position
    arr = np.transpose(arr, (2, 1, 0))

    return torch.from_numpy(arr)

In [ ]:
def save_samples(image_size: int):
    all_cells = []
    num_files = len(dfs)

    for idx in tqdm(range(num_files)):
        image_id = dfs.iloc[idx].ID
        labels = dfs.iloc[idx].Label
        red = read_img(image_id, "red", image_size)
        green = read_img(image_id, "green", image_size)
        blue = read_img(image_id, "blue", image_size)

        stacked_image = torch.stack([blue, green, red], axis=1)
        stacked_image = stacked_image[0,:,:,:]

        stacked_image = torch.transpose(stacked_image, dim0=2, dim1=0)

        fname = f'{image_id}.jpg'

        arr = stacked_image.cpu().numpy()
        im = Image.fromarray(arr, "RGB")
        torch

        im.save(path/f'train/{fname}', format="JPEG")

        all_cells.append({
            'image_id': image_id,
            'image_labels': labels
        })

    cell_df = pd.DataFrame(all_cells)
    cell_df.to_csv(path/'cell_df.csv', index=False)
    print(cell_df.head())


In [ ]:
save_samples(460)
